Sentiment analiza nad large movie reviews datasetom</br></br>


import os - za pristup fajlovaima</br>
import numpy - za računske operacije, te određene strukture poput nizova</br>
Tokenizer - pretvaranje teksta u nizove brojeva (uglavnom riječi koje se najčešće pojavljuju)</br>
Pad_sequences - neuralna mreža očekuje od ulaznih nizova da budu iste veličine, ovim omogućavamo jednake veličine (kraći se popunjavaju nulama, duži se skraćuju)</br>

Funkcija load_reviews učitava podatke iz dataseta</br>
S obzirom da je dataset formatiran kao glavni folder u kojem se nalaze dva podfoldera jedan za trening drugi za testne podatke</br>
U oba ova podfoldera imamao folder za pozitivne i negativne recenzije</br>
U ovim podfolderima svaki review je sačuvan kao poseban .txt file</br>
Potrebno je izvršiti ekstrakciju ovih podataka i smještanje u određene varijable kako bi mogli adekvatno nastaviti sa analizom</br></br>

Tokenizer je zadan da izrši pretvaranje 20000 najčešće spomenutih riječi u recenztijama u numeričku vrijednost</br>
Ovo se radi tako što u zavisnosti od količine ponavljanja riječ dobija indeks</br>
Texts.to.sequence potom kreira nizove takve da svaka rikeč ima svoj indeks (Film: 5, je: 2, odličan: 1)</br></br>

pad_sequences kao ranije navedeno sređuje ove nizove da budu iste veličine</br>

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def load_reviews(directory):
    texts, labels = [], []
    for label_type in ['neg', 'pos']:
        dir_path = os.path.join(directory, label_type)
        label = 0 if label_type == 'neg' else 1
        for fname in os.listdir(dir_path):
            if fname.endswith(".txt"):
                with open(os.path.join(dir_path, fname), encoding='utf-8') as f:
                    texts.append(f.read())
                    labels.append(label)
    return texts, labels

train_texts, train_labels = load_reviews('aclImdb/train')
test_texts, test_labels = load_reviews('aclImdb/test')

tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(train_texts)
x_train = tokenizer.texts_to_sequences(train_texts)
x_test = tokenizer.texts_to_sequences(test_texts)


maxlen = 300
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
y_train = np.array(train_labels)
y_test = np.array(test_labels)

In [ ]:
import pickle

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

Ovaj dataset je došao sa svojim zadanim vokabularom te ocjenom sentimenta koji smo također iskoristili za poboljšanje modela</br>
.vocab dakle sadrži ključne riječi</br>
dok imbdEr.txt sadži ocjene sentimenta</br></br>

vocab smještamo u niz kao i ocjene sentimenta</br></br>

word_to_sentiment spaja određenju riječ s njenom ocjenom te je u biti kreiran dictionary</br></br>

embedding vector je u biti niz feature-a svake riječi</br>
embedding matrica je u biti embedding vector za svaku riječ u jednoj strukturi</br>
U biti na kraju pomoću indeksiraih riječi i embedding matrice koja nam predstavlja težine dobijamo pripremljen skup za treniranje i analizu sentimenta</br>

In [ ]:
vocab_path = 'aclImdb/imdb.vocab'
sentiment_path = 'aclImdb/imdbEr.txt'

with open(vocab_path, encoding='utf-8') as f:
    vocab = [line.strip() for line in f]

sentiment_scores = np.loadtxt(sentiment_path)

word_to_sentiment = {word: sentiment_scores[i] for i, word in enumerate(vocab)}

embedding_dim = 128
embedding_matrix = np.zeros((20000, embedding_dim))

for word, i in tokenizer.word_index.items():
    if i >= 20000:
        continue
    sentiment = word_to_sentiment.get(word, 0.0)
    embedding_matrix[i] = np.full((embedding_dim,), sentiment)

In [ ]:
print("Sample training text:", train_texts[0][:500]) # temp

Sample training text: Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's bette


Sequential - način slaganja neurona (linearno) jedni za drugim u nizu</br>
Embedding sloj koji koristi prethodno pripremljene nizove i matricu, inicijalizira težine</br>
Dakle umjesto da model sam podešava početne težine mi mu dajemo težine koje smo izvukli iz dataseta što nam daje puno bolje performanse</br>
LSTM (Long-Short term memory) sloj procesira inpute iz prethodnog sloja</br>
LSTM je u biti vrta neuronske mreže tj. rekuralne neuronske mreže (RNN)</br>
Pogodan je za sentiment analizu zbog svoje efikasnoti za sekvencijalnim podacima tj. podacima koji ovise o kontekstu (o prethodnim i sljedećim članovima)</br>
Izabran je jer se dataset sastoji od 50000 podataka, LSTM pokazuje svoju efikasnot uglavnom na većim datasetovima</br>
Dropout parametri su podešeni da sprječe overfitting tj. prenaučenost modela</br>
Dense sloj je u potpunosti spojen s prethodnim slojem te koristi sigmoidnu aktivacijsku funkciju da nam da vjerovatno</br>
Rezultat teži 0 negativan sentiment</br>
Rezultat teži 1 pozitivan sentiment</br>
Loss je zadan na binary_crossentropy jer je efikasan za probleme binarne klasifikacije</br>
U našem slučaju to su dvije klase pozitivan i negativan sentiment</br>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential([
    Embedding(input_dim=20000, output_dim=embedding_dim,
              input_length=maxlen, weights=[embedding_matrix],
              trainable=True), 
    LSTM(128, dropout=0.4, recurrent_dropout=0.4),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,560,000 (9.77 MB)

 Trainable params: 2,560,000 (9.77 MB)

 Non-trainable params: 0 (0.00 B)

Early stopping se koristi kako bi se izbjegao overfitting, a također i za uštedu vremena</br>
U biti model je podešen da radi na 10 epoha tj. da 10 puta prođe kroz podatke te podesi težine i parametre</br>
Patience je 2 i na osnovu gubitka prekida trening i uzima slučaj najbolje epohe</br>
Validation split provjerava da li naš model dobro generalizira te da li je model sklon overfittingu</br>

In [5]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.2, callbacks=[early_stop])

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 154s 486ms/step - accuracy: 0.7673 - loss: 0.4929 - val_accuracy: 0.6030 - val_loss: 0.8534
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 141s 449ms/step - accuracy: 0.7932 - loss: 0.4515 - val_accuracy: 0.8134 - val_loss: 0.4829
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 144s 460ms/step - accuracy: 0.8224 - loss: 0.4108 - val_accuracy: 0.7568 - val_loss: 0.5154
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 146s 467ms/step - accuracy: 0.8521 - loss: 0.3439 - val_accuracy: 0.8158 - val_loss: 0.4367
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 144s 459ms/step - accuracy: 0.8798 - loss: 0.2904 - val_accuracy: 0.8236 - val_loss: 0.4802
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 129s 413ms/step - accuracy: 0.8983 - loss: 0.2599 - val_accuracy: 0.8072 - val_loss: 0.5021


Ostatak koda korišten je za evaluaciju i testiranje modela na eksternim podacima</br>

In [6]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 52ms/step - accuracy: 0.8993 - loss: 0.2469
Test Accuracy: 0.8662


In [8]:
def preprocess_input(text, tokenizer, maxlen=300):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=maxlen)
    return padded

In [ ]:
text = "Movie was bad."
input_data = preprocess_input(text, tokenizer)
prediction = model.predict(input_data)[0][0]

sentiment = "Positive 😀" if prediction >= 0.5 else "Negative 😞"
print(f"Predicted sentiment: {sentiment} (score: {prediction:.4f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
Predicted sentiment: Negative 😞 (score: 0.2343)


model je sačuvcan za upotrebu

In [10]:
model.save("sentiment_lstm_model.h5")